In [1]:
#Importing libraries needed
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pyplot
%matplotlib inline
import numpy as np
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

## Zillow Data

### Median home value from Kaggle

Just zillow 
-create a dataframe from all of the hurricane data that has target variable (percent increase and yes or no), value of homes 4 months before hurricane, city names, which hurricane, and size ranks

-use statsmodel or scikit learn to do logistic regression on categorical column 4 month before the hurricane (predictnig the value)

## Data Exploration

In [2]:
#opening the dataset to create a dataframe for logistic regression 
value = pd.read_csv(r'data\Sale_Prices_City.csv')
value.head()

,Unnamed: 0,RegionID,RegionName,StateName,SizeRank,2015-10,2015-11,2015-12,2016-01,2016-02,...,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10
0,12,25290,Jacksonville,Florida,13,98900.0,98700.0,98900.0,100100.0,102100.0,...,141900,146300,147400,153300,156700,164300,171000,173000,172500,170600.0
1,20,13121,Orlando,Florida,21,169900.0,172800.0,174700.0,176700.0,176400.0,...,239900,241000,238700,239100,240100,244200,244900,241900,241500,243600.0
2,38,12700,Miami,Florida,39,286500.0,291800.0,292100.0,289200.0,291800.0,...,308300,307300,309500,316500,318700,321700,323600,320400,326100,333500.0
3,53,41176,Tampa,Florida,54,158400.0,161400.0,157800.0,167800.0,168500.0,...,229800,227600,219900,220700,222100,229000,228400,234000,232800,240500.0
4,71,19533,Naples,Florida,72,275100.0,281300.0,287600.0,297600.0,290800.0,...,310700,309000,310500,313000,319500,319700,323100,320600,322800,321800.0


## Data Scrubbing

In [3]:
#Renaming columns 4 months before and after each hurricane
#These values will be used to do logistic regression
#Matthew (10/06-08/2016): Before 2016-06 After 2017-02
value['BMatthew'] = value['2016-06']
value['AMatthew'] = value['2017-02']

#Irma (09/10-11/2017): Before 2017-05 After 2018-01
value['BIrma'] = value['2017-05']
value['AIrma'] = value['2018-01']

#Michael (10/09-11/2018): Before 2018-06 After 2019-02 
value['BMichael'] = value['2018-06']
value['AMichael'] = value['2019-02']

#dropping unnecessary columns 
#droppping the rest of the value columns
value = value.loc[:, ~value.columns.str.startswith('2')]

#dropping unnamed: 0 and regionID because they are unique identifiers
#dropping StateName because Florida is the only state being worked with
value.drop(['Unnamed: 0','RegionID', 'StateName'], axis=1, inplace=True)

#Checking it out
value.head()

,RegionName,SizeRank,BMatthew,AMatthew,BIrma,AIrma,BMichael,AMichael
0,Jacksonville,13,111700.0,128500.0,129200.0,133900.0,140700,146300
1,Orlando,21,179800.0,191400.0,196000.0,213000.0,234800,241000
2,Miami,39,286300.0,301100.0,304100.0,312900.0,315500,307300
3,Tampa,54,187300.0,187500.0,185300.0,205100.0,211600,227600
4,Naples,72,283400.0,292600.0,299600.0,295500.0,311500,309000


In [4]:
#let's create a dataframe for each hurricane and then concat them together

#creating dataframe for hurricane Matthew (1)
df_Ma = value[['RegionName', 'SizeRank', 'BMatthew', 'AMatthew']]
#HurricaneName will be '1' 
df_Ma['HurricaneName'] = '1'
df_Ma = df_Ma.rename(columns={"BMatthew": "Before", "AMatthew": "After"})

#creating dataframe for hurricane Irma (2)
df_I = value[['RegionName', 'SizeRank', 'BIrma', 'AIrma']]
#HurricaneName will be '2' 
df_I['HurricaneName'] = '2'
df_I = df_I.rename(columns={"BIrma": "Before", "AIrma": "After"})

#creating dataframe for hurricane Michael (3)
df_Mi = value[['RegionName', 'SizeRank', 'BMichael', 'AMichael']]
#HurricaneName will be '3' 
df_Mi['HurricaneName'] = '3'
df_Mi = df_Mi.rename(columns={"BMichael": "Before", "AMichael": "After"})

#concating the three dataframes into one 
df = pd.concat([df_Ma, df_I, df_Mi], ignore_index=True)

#renaming 'RegionName' to 'City'
df = df.rename(columns={"RegionName":"City"})

#taking a look
df.head()

,City,SizeRank,Before,After,HurricaneName
0,Jacksonville,13,111700.0,128500.0,1
1,Orlando,21,179800.0,191400.0,1
2,Miami,39,286300.0,301100.0,1
3,Tampa,54,187300.0,187500.0,1
4,Naples,72,283400.0,292600.0,1


## Data Exploration 

Let's get an idea of what home value increase looks like from 4 months before and after the hurricanes. This will inform the cutoff value we use when modeling. Our Q3 value is just about 10% for all three hurricanes, so this will be the value used for cutoff.  

In [5]:
#percent = (after - before/before)*100

#finding percent change for Hurricane Matthew
df['Percent'] = (df['After'] - df['Before'])/df['Before']*100

#Q3 ~10% 
df.describe()

,SizeRank,Before,After,Percent
count,831.000000,791.000000,813.000000,791.000000
mean,1711.162455,229482.806574,241690.897909,5.961250
std,1149.142889,104078.880056,109101.900399,7.948774
min,13.000000,34100.000000,35600.000000,-21.457404
25%,691.000000,162200.000000,171200.000000,1.468005
50%,1587.000000,200800.000000,215300.000000,5.689001
75%,2715.000000,268900.000000,280600.000000,9.992094
max,3723.000000,735400.000000,836600.000000,42.339008


## Data Engineering

Using the 10% cutoff value a column will be engineered stating if 'True' there was a 10% of more increase in value, or 'False' there was less than a 10% increase in value. 

In [6]:
#1 = true, there was a 10% more increase in value 
#0 = false, there was not a 10% or more increase in value
df['Increase'] = np.where(df['Percent'] >= 10, 1, 0)
df.head()

,City,SizeRank,Before,After,HurricaneName,Percent,Increase
0,Jacksonville,13,111700.0,128500.0,1,15.040286,1
1,Orlando,21,179800.0,191400.0,1,6.451613,0
2,Miami,39,286300.0,301100.0,1,5.169403,0
3,Tampa,54,187300.0,187500.0,1,0.106781,0
4,Naples,72,283400.0,292600.0,1,3.246295,0


In [7]:
#Dropping any missing values
df = df.dropna()
df.isnull().sum()

City             0
SizeRank         0
Before           0
After            0
HurricaneName    0
Percent          0
Increase         0
dtype: int64

In [8]:
df

,City,SizeRank,Before,After,HurricaneName,Percent,Increase
0,Jacksonville,13,111700.0,128500.0,1,15.040286,1
1,Orlando,21,179800.0,191400.0,1,6.451613,0
2,Miami,39,286300.0,301100.0,1,5.169403,0
3,Tampa,54,187300.0,187500.0,1,0.106781,0
4,Naples,72,283400.0,292600.0,1,3.246295,0
...,...,...,...,...,...,...,...
826,Tierra Verde,3703,484700.0,545300.0,3,12.502579,1
827,Placida,3708,301600.0,353400.0,3,17.175066,1
828,Daytona Beach Shores,3711,253700.0,288900.0,3,13.874655,1
829,Holmes Beach,3713,618200.0,720500.0,3,16.548043,1


### Converting Categorical Data

In [ ]:
#one hot encoding 'RegionName'
origin = df['City']
origin_series = pd.Series(origin)
df['City'] = origin_series.astype('category')
df['City'] = df['City'].cat.codes

## Modeling 

In [ ]:
#Defining  as our target (y)
#Defining the rest of the columns as our features (X)

y = df['Increase']
X = df.drop(columns='Increase', axis=1)

### Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

#setting up our train/test split
#stratifying the data due to the large imablance in target classes 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=56, stratify=y)
# Check size of train and test sets
print(len(X_train), len(X_test), len(y_train), len(y_test))

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(fit_intercept=False, C=1e12, solver='liblinear')
model_log = logreg.fit(X_train, y_train)
model_log

In [ ]:
y_hat_train = logreg.predict(X_train)

train_residuals = np.abs(y_train - y_hat_train)
print(pd.Series(train_residuals, name="Residuals (counts)").value_counts())
print()
print(pd.Series(train_residuals, name="Residuals (proportions)").value_counts(normalize=True))

In [ ]:
y_hat_test = logreg.predict(X_test)

test_residuals = np.abs(y_test - y_hat_test)
print(pd.Series(test_residuals, name="Residuals (counts)").value_counts())
print()
print(pd.Series(test_residuals, name="Residuals (proportions)").value_counts(normalize=True))

In [ ]:
df